In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [17]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
import malaya

model = malaya.translation.ms_en.transformer(model = 'small-bigbird')

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [5]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
left = [unidecode(s) for s in ms]
right = [unidecode(s) for s in eng]

In [6]:
len(ms), len(eng)

(998, 998)

In [10]:
from tqdm import tqdm

batch_size = 8

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:16<00:00,  7.49it/s]


In [11]:
len(results)

998

In [12]:
import re

def bigbird_postprocessing(string):
    return re.sub(r'([!,.?/;\'])\1+', r'\1 ', string).strip()

filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(bigbird_postprocessing(r))
        filtered_right.append(right[no])

In [13]:
refs = [filtered_right]
sys = filtered_left

In [14]:
r = bleu.corpus_score(sys, refs)

In [15]:
r.__dict__

{'name': 'BLEU',
 'score': 36.33139297963485,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '68.9/44.1/30.3/21.2 (BP = 0.973 ratio = 0.973 hyp_len = 22937 ref_len = 23570)',
 'bp': 0.9727799954961814,
 'counts': [15799, 9677, 6341, 4219],
 'totals': [22937, 21940, 20943, 19946],
 'sys_len': 22937,
 'ref_len': 23570,
 'precisions': [68.87997558529887,
  44.10665451230629,
  30.277419662894523,
  21.152110698886993],
 'prec_str': '68.9/44.1/30.3/21.2',
 'ratio': 0.9731438268985999}

In [18]:
chrf.corpus_score(sys, refs)

chrF2++ = 60.57